<a href="https://colab.research.google.com/github/jgkorman/Statistical-Analysis-In-Python/blob/main/Assignment_6_PDLA_Korman_JamesG_ICPSR_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ICPSR PDLA Assignment 6**
Author: Korman, James, G



# **Directions**


This assignment will use five waves of the British Election Study leading up to the
2015 General Election (BESdatasubset.dta, BESdatasubsetforR.dta). Specifically, we will
look at the relationship between intention to vote for the incumbent Conservative party
and evaluations of the performance of the economy. The variable VoteCon is coded ’1’ if
the respondent indicated they intend on voting for the Conservative party candidate and
’0’ otherwise. The variable changeEconomy contains the responses to the question: "Do
you think that each of the following are getting better, getting worse or staying about the
same? Economy". Response options are coded: 1 Getting a lot worse; 2 Getting a little
worse; 3 Staying about the same; 4 Getting a little better; 5 Getting a lot better.

## **Reading in the Data & Descriptive Statistics**

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

import scipy.stats as st
import statsmodels.api as sm
import statsmodels.graphics.tsaplots as tsap
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_white


In [11]:
dtafile = '/content/drive/MyDrive/Delaware/ICPSR_2022/Panel Data And Longitudinal Analysis/Assignments/Assignment 6/Copy of BESdatasubsetforR.dta'

df = pd.read_stata(dtafile)
df['time'] = df['t']
df['lag_VoteCon'] = df['VoteCon'].shift(1)
df['lag_time'] = df['time'].shift(1)
df = df.set_index(['id', 't'])
df

generalElectionVote            changeEconomy  VoteCon  \
id    t                                                         
3     1              Labour   Staying about the same      0.0   
      2              Labour   Staying about the same      0.0   
      3              Labour   Staying about the same      0.0   
      4              Labour   Staying about the same      0.0   
      5              Labour                      NaN      0.0   
...                     ...                      ...      ...   
30590 1                 NaN   Staying about the same      0.0   
      2                 NaN   Staying about the same      0.0   
      3                 NaN  Getting a little better      0.0   
      4                 NaN   Staying about the same      0.0   
      5                 NaN                      NaN      0.0   

         _est_pooled_vote  _est_fe_vote  L_changeEconomy  wave1  wave2  wave3  \
id    t                                                                         
3     1                 0             0              NaN      1      0      0   
      2                 1             1              3.0      0      1      0   
      3                 1             1              3.0      0      0      1   
      4                 1             1              3.0      0      0      0   
      5                 1             1              3.0      0      0      0   
...                   ...           ...              ...    ...    ...    ...   
30590 1                 0             0              NaN      1      0      0   
      2                 0             0              3.0      0      1      0   
      3                 0             0              3.0      0      0      1   
      4                 0             0              4.0      0      0      0   
      5                 0             0              3.0      0      0      0   

         wave4  wave5  _est_gmm_approve  _est_gmm_vote  _est_qmlfe_vote  \
id    t                                                                   
3     1      0      0                 0              0                0   
      2      0      0                 1              1                0   
      3      0      0                 1              1                1   
      4      1      0                 1              1                1   
      5      0      1                 1              1                1   
...        ...    ...               ...            ...              ...   
30590 1      0      0                 0              0                0   
      2      0      0                 0              0                0   
      3      0      0                 0              0                0   
      4      1      0                 0              0                0   
      5      0      1                 0              0                0   

         L_generalElectionVote  time  lag_VoteCon  lag_time  
id    t                                                      
3     1                    NaN     1          NaN       NaN  
      2                    2.0     2          0.0       1.0  
      3                    2.0     3          0.0       2.0  
      4                    2.0     4          0.0       3.0  
      5                    2.0     5          0.0       4.0  
...                        ...   ...          ...       ...  
30590 1                    NaN     1          0.0       5.0  
      2                    NaN     2          0.0       1.0  
      3                    NaN     3          0.0       2.0  
      4                    NaN     4          0.0       3.0  
      5                    NaN     5          0.0       4.0  

[82500 rows x 18 columns]

# **1. Question 1**

Using OLS, estimate the following pooled LDV model with time specific effects: 

**VoteConi,t = aVoteConi,t-1 + b1changeEconomyi,t + tt + ei,t.** 

Report the short-run effect. Calculate and
test the long-run effect. What do you find?

In [12]:
!pip install linearmodels
from linearmodels import PooledOLS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:

endog = df['VoteCon']
exog = df[['lag_VoteCon', 'changeEconomy', 'time', 'lag_time']]
exog = sm.tools.tools.add_constant(exog)
mod = PooledOLS(endog, exog)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res.summary)


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  # put in order ctt
/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:37: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PooledOLS Estimation Summary                          
Dep. Variable:                VoteCon   R-squared:                        0.4901
Estimator:                  PooledOLS   R-squared (Between):              0.7839
No. Observations:               82499   R-squared (Within):              -0.9409
Date:                Fri, Aug 05 2022   R-squared (Overall):              0.4901
Time:                        21:42:02   Log-likelihood                -2.134e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   1.133e+04
Entities:                       16500   P-value                           0.0000
Avg Obs:                       4.9999   Distribution:                 F(7,82491)
Min Obs:                       4.0000                                           
Max Obs:                       5.0000   F-statistic (robust):          1.034e+04
                            

Our pooled OLS model that estimates a probability linear model showes that the lag of Voting conservative is a highly statistically significant predictor of voting conservative in the short run effect -- ie if you voted conservative the year before, its a good predictor that you will vote conservative again. Our categorically ordered variable for 'change economy' that measures attitudes towards feelings of the economy from 1-5 (least to best) (with 1 (getting a lot worse left out as our reference category) shows that if the economy is getting worse, is a statistically signifiacnt predictor that one will note vote cosnervative. However, if poeple genuninely feel that if the economy is improving, the improving economy is associated with voting conservative and is positive and highly staitsitcally significant in the short-run effects, and the magnitude of the effects becomes greater the more that respondents feel better about the economuy suggesting the more the economy improves, the more likely folks are to vote conservative. Time in the short run effect is highly statistically significant, suggesting to us that as time goes by, people are more likely to vote conservative. The lag of time at t-1 is also highly statistiaclly significant which suggests to us that time has "sticky effects" and if you voted conservative in past time, there's a good probability one will do so again. 

For the long run effect of the lag of vote conservative -- it is -0.0739 / 1 - 0.5976 = -.18 which suggests to us the long run effect of voting conservative is negative meaning if you vote conservatie today, you'll be less likely to vote conservative tomorrow. 

The long run effect of 'time' on 'vote conservative' is: 0.0303 + 0.0061 / 1 -0.5976 = 0.090 suggesting to us the total cumulative effect of time on voting conservatie is substantial. 

# **2. Question 2** 

Using OLS, estimate the following fixed-effects LDV model with time specific effects:

**VoteConi,t = aVoteConi,t-1 + b1changeEconomyi,t + tt + hi + ei,t** 

Report the short-run
effect. Calculate and test the long-run effect. What do you find?

In [14]:
# importing the fixed effects estimator 
from linearmodels import PanelOLS
mod = PanelOLS(endog, exog, time_effects=True, drop_absorbed=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res.summary)

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:37: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1833: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

time, lag_time

  AbsorbingEffectWarning,


                          PanelOLS Estimation Summary                           
Dep. Variable:                VoteCon   R-squared:                        0.4918
Estimator:                   PanelOLS   R-squared (Between):              0.7835
No. Observations:               82499   R-squared (Within):              -1.0525
Date:                Fri, Aug 05 2022   R-squared (Overall):              0.4708
Time:                        21:42:03   Log-likelihood                -2.111e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   1.597e+04
Entities:                       16500   P-value                           0.0000
Avg Obs:                       4.9999   Distribution:                 F(5,82489)
Min Obs:                       4.0000                                           
Max Obs:                       5.0000   F-statistic (robust):          1.132e+04
                            

For the fixed effects model estimated directly above, in contrast to the Pooled OLS model, explained roughly 47% of the variation in the DV relative to 49% of the variation in the DV as was explained via the Pooled OLS model. Time fixed effects were reported to ensure that the impacts of time are not confounding our results to control for the unobserved hetereogenieties that are present to all units over time. Our lagged DV of vote conservative again was statisitcally significant and positive suggesting to us that if you vote conservative one year, you're most likely going to vote conservative again the next year. Our categorical factored variable for change economy demonstrates that improvements to the economy from "getting a lot worse" which is the reference category (left out of the model) all have a positive and statistically significant impact on voting conservative with the magntitude of those short-run effects becoming greater the stronger the economy is improving. 

The long-run effect for lagged DV = -0.0015 / (1 - .5923) = -0.00367 suggesting to us the long run effect of voting conservative over time will lead to a decrease in voting conservative in the future. 

# **3. Question 3**. 

Estimate the same two models as Q1 using one of the GMM estimators (I recommend
using a Stata command). Indicate which one you used. Apply and report the autocorrelation
and Sargan tests. Don’t worry if your model specificatiomn does not pass these
tests – just report the results. How do your GMM estimates compare to the Pooled and
OLS-FE estimates?

In [6]:
!pip install pydynpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd
from  pydynpd import regression

In [8]:
df.columns

Index(['generalElectionVote', 'changeEconomy', 'VoteCon', '_est_pooled_vote',
       '_est_fe_vote', 'L_changeEconomy', 'wave1', 'wave2', 'wave3', 'wave4',
       'wave5', '_est_gmm_approve', '_est_gmm_vote', '_est_qmlfe_vote',
       'L_generalElectionVote', 'time', 'lag_VoteCon', 'lag_time'],
      dtype='object')

In [25]:
command_str= 'VoteCon lag_VoteCon changeEconomy | gmm(lag_VoteCon, 1:2) gmm(changeEconomy, 1:.) | nolevel collapse'


In [26]:
df = df.reset_index()
df

,level_0,index,id,t,generalElectionVote,changeEconomy,VoteCon,_est_pooled_vote,_est_fe_vote,L_changeEconomy,...,wave3,wave4,wave5,_est_gmm_approve,_est_gmm_vote,_est_qmlfe_vote,L_generalElectionVote,time,lag_VoteCon,lag_time
0,0,0,3,1,Labour,Staying about the same,0.0,0,0,NaN,...,0,0,0,0,0,0,NaN,1,NaN,NaN
1,1,1,3,2,Labour,Staying about the same,0.0,1,1,3.0,...,0,0,0,1,1,0,2.0,2,0.0,1.0
2,2,2,3,3,Labour,Staying about the same,0.0,1,1,3.0,...,1,0,0,1,1,1,2.0,3,0.0,2.0
3,3,3,3,4,Labour,Staying about the same,0.0,1,1,3.0,...,0,1,0,1,1,1,2.0,4,0.0,3.0
4,4,4,3,5,Labour,NaN,0.0,1,1,3.0,...,0,0,1,1,1,1,2.0,5,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82495,82495,82495,30590,1,NaN,Staying about the same,0.0,0,0,NaN,...,0,0,0,0,0,0,NaN,1,0.0,5.0
82496,82496,82496,30590,2,NaN,Staying about the same,0.0,0,0,3.0,...,0,0,0,0,0,0,NaN,2,0.0,1.0
82497,82497,82497,30590,3,NaN,Getting a little better,0.0,0,0,3.0,...,1,0,0,0,0,0,NaN,3,0.0,2.0
82498,82498,82498,30590,4,NaN,Staying about the same,0.0,0,0,4.0,...,0,1,0,0,0,0,NaN,4,0.0,3.0


In [27]:
mydpd = regression.abond(command_str, df, ['id', 't'])
mydpd

IndexError: ignored

For the fixed effects model estimated above, it only explained about 13% of the variation in log wages from the adjusted R squared -- this is plausible as it is the most restricted model in the sense that it only uses within variance to derive the estimates. The fixed effects model estiamted above also includes both time and entity effects to control for unobserved heterogenieities both across units, and over time. For the fixed effects model, the variable "age" is not statistically significant as opposed to the pooled OLS and random effects model suggesting to us age has no effect on log wages in eitehr direction. Being married(msp) again isn't statistically significant. Meanwhile, total experience again was positive and highly stastistically signifiacnt suggesting to us that total experience is a very salient predictor of log wages. 

Overall, python is unabe to estimate the GMM estimator along with the QML and OPM estimators. This class of estimators still needs to be improved upon and worked on. However, the coefficients on the pooled OLS model and the dynamic fixed effects model -- they are exactly the same, and this is most likely a result of the fact that both of these estimators are using the within variation to a great extent to estimate these factors. However, since the Pooled OLS model is using both within and between variation -- it explained more of the variation in the DV relative to the fixed effects model which really employs only within-variation to make the estimates.